In [ ]:
# load libraries
import torch
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import os
import pandas as pd
import glob
from segment_anything import SamPredictor, sam_model_registry
from scipy.stats import gaussian_kde
import mplcursors
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

In [3]:
# define mask and box functions

def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   
    
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))

In [ ]:
# Model
weight = #path to weight from Etienne David's repo
model = torch.hub.load("ultralytics/yolov5", "custom", path=weight)  # custom model

img_path= #path to image
results = model(img_path)

# Then we transform into boxes as an input for SAM

inputs = results.pandas().xyxy[0][["xmin", "ymin", "xmax", "ymax"]].values.astype(int)

# set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# load model
sam = sam_model_registry["default"](checkpoint= #path to SAM checkpoint)
predictor = SamPredictor(sam)

img_arr = cv2.imread(img_path)
img_arr = cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB)

predictor.set_image(img_arr)
masks = []
for input_ in inputs:
    mask, _, _ = predictor.predict(
        point_coords=None,
        point_labels=None,
        box=input_,
        multimask_output=False,
    ) 
    masks.append(mask)

%matplotlib inline
import matplotlib.pyplot as plt
plt.imshow(img_arr)

for input_,mask in zip(inputs,masks):
    show_mask(mask, plt.gca(), random_color=True)
plt.show()

combined_mask = np.zeros((masks[0].shape[1], masks[0].shape[2]), dtype=np.uint8)
for mask in masks:
    combined_mask = np.logical_or(combined_mask, mask[0])

# Convert the segmentation mask to a binary mask
binary_mask = np.where(combined_mask > 0.5, 1, 0)

#create a background with NA values
na_background = np.full_like(img_arr, np.nan)
#create a function to divide the red band by the green band to create a new image
img_na = na_background * (1 - binary_mask[..., np.newaxis]) + img_arr * binary_mask[..., np.newaxis]
s_r = img_na[:,:,0]
s_g = img_na[:,:,1]
s_b = img_na[:,:,2]

#performing the red/green ratio calculation
rg = s_r / (s_g + s_r)

rg[~combined_mask] = np.nan
# Create a new figure and arrange the histogram and exgr image side by side with a colorbar
plt.figure(figsize=(15, 10))

rg_min = 0.2
rg_max = 0.6

# Plot the rg image on the left side
plt.subplot(1, 2, 1)
plt.imshow(rg, cmap='Spectral', vmin=rg_min, vmax=rg_max)
plt.colorbar()

# Filter out invalid values (inf and NaN) from the exgr array
valid_rg = rg.ravel()[np.isfinite(rg.ravel())]

# Plot the smoothed density histogram on the right side
plt.subplot(1, 2, 2)

kde = gaussian_kde(valid_rg)

# Set the range of values for the x-axis
x_vals = np.linspace(rg_min, rg_max, 500)

# Obtain colors from the 'Spectral' colormap for the histogram bars
colors = plt.get_cmap('Spectral')(np.linspace(0, 1, len(x_vals)))

# Plot the histogram as bars with colors based on x-value
plt.bar(x_vals, kde(x_vals), width=(x_vals[1]-x_vals[0]), color=colors)

# Add labels and title
plt.xlabel('RG Values')
plt.ylabel('Density')
plt.title('Smoothed Density Histogram')

# Set the x-axis limits to the specified range
plt.xlim(rg_min, rg_max)

# Show the plot
plt.show()

In [ ]:
#count the total number of pixels within combined mask
total_pixels = np.count_nonzero(combined_mask)
print(total_pixels)
#count the number of pixels within the combined mask that have a red/green ratio less than the threshold
red_pixels = np.count_nonzero(rg > 0.49) #change threshold here
print(red_pixels)

#calculate the severity of the disease
severity = (red_pixels/total_pixels)*100
print(severity)

In [ ]:
# determine the accuracy of the rg transformation severity estimations through stratified random sampling

# Set the pixels within the mask to binary based on the rg value
binary_mask = np.where((combined_mask > 0) & (rg > 0.49), 1, 0) #change threshold here

# visualize the binary mask
plt.imshow(binary_mask, cmap='gray')
plt.show()

# Get the indices of all non-zero pixels in the binary mask
indices = np.argwhere(combined_mask > 0)

# Get the values of each pixel in the binary mask
values = binary_mask[indices[:, 0], indices[:, 1]]

# Perform stratified random sampling to select 30 indices with a proportional amount of 0s and 1s
sss = StratifiedShuffleSplit(n_splits=1, test_size=30, random_state=42)
train_index, test_index = next(sss.split(indices, values))

# Get the selected indices and their corresponding values
selected_indices = indices[test_index]
selected_values = values[test_index]

# Create a DataFrame to store the position and value of each selected pixel
df = pd.DataFrame(columns=['Row', 'Column', 'Value'])

# Loop through each selected index and record its position and value in a list of dictionaries
data = []
for index, value in zip(selected_indices, selected_values):
    row, col = index
    data.append({'Row': row, 'Column': col, 'Value': value})

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data, columns=['Row', 'Column', 'Value'])

# Print the DataFrame to the console
print(df)

In [ ]:
# Load the original RGB image
img = cv2.imread(#path to image)

# Set the row and column numbers for the desired pixel
row_num = #row number
col_num = #column number

# Highlight the desired pixel in the image
img_highlighted = img.copy()
cv2.circle(img_highlighted, (col_num, row_num), 5, (0, 0, 255), -1)

# Display the highlighted image using Matplotlib
plt.imshow(cv2.cvtColor(img_highlighted, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
plt.style.use('default')

# Load the Excel file into a DataFrame
df = pd.read_excel(# path to excel file with estimated and observed pixel values)

# Get the observed and predicted class labels from the DataFrame
observed = df['Visual']
predicted = df['Value']

# Create a confusion matrix
cm = confusion_matrix(observed, predicted)

# Calculate accuracy, recall, precision, and F1 scores
accuracy = accuracy_score(observed, predicted)
recall = recall_score(observed, predicted, average='weighted')
precision = precision_score(observed, predicted, average='weighted')
f1 = f1_score(observed, predicted, average='weighted')

# Display the confusion matrix as a heatmap using Matplotlib
sns.heatmap(cm, annot=True, cmap='Blues', annot_kws={'size': 15})
plt.xlabel('Predicted Class', fontsize=15)
plt.ylabel('Observed Class', fontsize=15)
plt.title('Confusion Matrix', fontsize=15)
plt.show()

# Print the scores to the console
print('Accuracy:', accuracy)
print('Recall:', recall)
print('Precision:', precision)
print('F1 Score:', f1)